In [72]:
import pandas as pd

In [76]:
df = pd.read_parquet(path='geo_data/complete_data.parquet')

In [77]:
headers = ['zone_name','country_code','abbreviation','time_start','gmt_offset','dst']
tst_df = pd.read_csv('geo_data/tz_db/time_zone.csv', names=headers, 
                     usecols=['zone_name', 'country_code', 'abbreviation', 'gmt_offset'])

In [102]:
country = pd.read_csv('geo_data/tz_db/country.csv', names=['country_code', 'country'])
res_df = tst_df.merge(right=country, on='country_code') \
.drop(columns=['country_code']) \
.rename(columns=dict(zone_name='timezone')) \
.drop_duplicates() \
.reset_index(drop=True)

In [103]:
temp_offset = round(res_df.gmt_offset / 60).astype('int').map(lambda x: divmod(x, 60))

In [104]:
def parse_offset(x: tuple):
    s = ''
    h, m = x
    if h < 0:
        s += '-'
    else:
        s += '+'
    if abs(h) < 10:
        s += '0'
    s += f'{abs(h)}:'
    if abs(m) < 10:
        s += '0'
    s += str(m)
    return s

In [105]:
res_df['offset_str'] = temp_offset.map(parse_offset).astype('string')

In [107]:
res_df = res_df.astype(dict(timezone='string', abbreviation='string', country='string'))

In [99]:
right = pd.read_parquet('geo_data/complete_data.parquet')

In [151]:
full_res = res_df.merge(right=right, on='timezone') \
.drop(columns=['utc_offset', 'tz_name', 'country_x'])
first = ['name', 'population']
rest = [item for item in full_res if item not in first]
full_res = full_res.reindex(columns=(first + rest)) \
.astype(dict(timezone='category', abbreviation='category', offset_str='category')) \
.rename(columns=dict(country_y='country'))

In [152]:
full_res.to_parquet(path='geo_data/tz_db/tz_dictionary.parquet')